In [ ]:
import os 
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter

from sklearn.model_selection import train_test_split

# Own library imports
from vecopsciml.utils import TensOps
from vecopsciml.operators.zero_order import Mx, My
from vecopsciml.kernels.derivative import DerivativeKernels

# Function from this project
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop
from utils.checkpoints import load_results

# Import model
from architectures.autoencoder import Autoencoder

In [ ]:
# Dataset
dataset = 'non_linear'
N_data = 10
noise = 0

data_name = dataset + '_' + str(N_data) + '_' + str(noise)
print(data_name)

In [ ]:
# Model
model = 'autoencoder'
n_modes = 5

model_name = model + '_model_' + str(n_modes)
print(model_name)

In [ ]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "../../"))
DATA_PATH = os.path.join(ROOT_PATH, r'data/', data_name, data_name) + '.pkl'
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/', data_name)

MODEL_RESULTS_AE_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name) + '_AE'
MODEL_RESULTS_PGNNIV_PATH = os.path.join(ROOT_PATH, r'results/', data_name, model_name) + '_NN'

# Create folders (if necessary)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_AE_PATH)
create_folder(MODEL_RESULTS_PGNNIV_PATH)

In [ ]:
# Load dataset
dataset = load_data(DATA_PATH)

In [ ]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [ ]:
DEVICE = torch.device("cpu")

print(f"Using device: {DEVICE}")

In [ ]:
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = torch.Tensor(dataset['y_train']).unsqueeze(1)
K_train = torch.tensor(dataset['k_train']).unsqueeze(1)
f_train = torch.tensor(dataset['f_train']).unsqueeze(1)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.Tensor(dataset['y_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val']).unsqueeze(1).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

print("Train dataset length:", len(X_train))
print("Validation dataset length:", len(X_val))

In [ ]:
N_data_AE = len(X_train)//4
N_data_NN = len(X_train) - len(X_train)//4
prop_data_NN = 1 - N_data_AE/(N_data_NN + N_data_AE)

print("Dataset length for the autoencoder:", N_data_AE)
print("Dataset length for the PGNNIV:", N_data_NN)

X_AE, X_NN, y_AE, y_NN, K_AE, K_NN, f_AE, f_NN = train_test_split(X_train, y_train, K_train, f_train, test_size=prop_data_NN, random_state=42)

In [ ]:
y_train_AE, y_test_AE = train_test_split(y_AE, test_size=0.2, random_state=42)

y_train_AE = TensOps(y_train_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)
y_test_AE = TensOps(y_test_AE.requires_grad_(True).to(DEVICE), space_dimension=2, contravariance=0, covariance=0)

In [ ]:
autoencoder_input_shape = y_train_AE.values[0].shape
latent_space_dim = [20, 10, n_modes, 10, 20]
autoencoder_output_shape = y_train_AE.values[0].shape

model = Autoencoder(autoencoder_input_shape, latent_space_dim, autoencoder_output_shape).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

model, optimizer, lists = load_results(model, optimizer, MODEL_RESULTS_AE_PATH, map_location=torch.device('cpu'))

In [ ]:
train_total_loss_list = lists['train_total_loss_list']
test_total_loss_list = lists['test_total_loss_list']
time_list = lists['time_list'] 

In [ ]:
def smooth_curve(data, window_size=100):
    window = np.ones(window_size) / window_size
    return np.convolve(data, window, mode='valid')

def cm_to_in(cm):
    return cm * 0.393701

def normalize_list(lst):
    max_value = np.max(lst)
    return [x / max_value for x in lst]

linewidth = 1.5  
title_fontsize = 14  
label_fontsize = 14  
legend_fontsize = 12 
tick_fontsize = 11  

# plt.rc('text', usetex=True)
plt.rc('font', family='serif')

posX = cm_to_in(10) 
posY = cm_to_in(10)
width = cm_to_in(12)
height = cm_to_in(8)

color = [0.1, 0, 0.8]
subplot_adjust_left = cm_to_in(0.15)
subplot_adjust_bottom = cm_to_in(0.15)

In [ ]:
plt.figure(figsize=(width, height))

plt.plot(smooth_curve(train_total_loss_list), label='Total MSE train', color='blue', linestyle='-')
plt.plot(smooth_curve(test_total_loss_list), label='Total MSE test', color='red', linestyle='--')

plt.xlabel('Iteration', fontsize=label_fontsize)
plt.ylabel('Loss', fontsize=label_fontsize)

plt.grid(True)
plt.legend(loc='lower left', fontsize=legend_fontsize)
plt.tick_params(axis='both', which='major', labelsize=tick_fontsize)

plt.xscale('log')
plt.yscale('log')
plt.xlim(left=1) 

plt.show()

In [ ]:
def relative_error_stats(validation, prediction, dx=dx, dy=dy):
    validation = validation.numpy()
    prediction = prediction.numpy()

    prediction_error = np.sqrt((np.trapz(np.trapz((validation - prediction)**2, dx=dy), dx=dx) /
                                np.trapz(np.trapz((validation)**2, dx=dy), dx=dx)))

    minimum = np.min(prediction_error)
    maximum = np.max(prediction_error)
    first_quartile = np.percentile(prediction_error, 25)
    median = np.percentile(prediction_error, 50)
    third_quartile = np.percentile(prediction_error, 75)

    # Print the results
    print(f"Minimum: {minimum:.2e}")
    print(f"First quartile (Q1): {first_quartile:.2e}")
    print(f"Median (Q2): {median:.2e}")
    print(f"Third quartile (Q3): {third_quartile:.2e}")
    print(f"Maximum: {maximum:.2e}")


def relative_error_return_Q(validation, prediction, dx=dx, dy=dy):
    validation = validation.numpy()
    prediction = prediction.numpy()

    prediction_error = np.sqrt((np.trapz(np.trapz((validation - prediction)**2, dx=dy), dx=dx)/
                                np.trapz(np.trapz((validation)**2, dx=dy), dx=dx)))
    third_quartile = np.percentile(prediction_error, 75)

    Q_bool = prediction_error <= third_quartile

    return Q_bool

In [ ]:
X_val = y_val.values

y_pred = TensOps(model(X_val), space_dimension=2, contravariance=0, covariance=0)

### Solution $u(x,y)$

In [ ]:
u_validation = y_val.values.detach()
u_prediction = y_pred.values.detach()

relative_error_stats(u_validation, u_prediction)

In [ ]:
plt.figure(figsize=(height*1.2, height))

u_diff = torch.mean(torch.abs(u_prediction - u_validation), axis=0).squeeze()
plt.imshow(u_diff, interpolation='bicubic', extent=[0, 1, 0, 1])

cbar = plt.colorbar(fraction=0.046, pad=0.04)
cbar.formatter = ScalarFormatter(useMathText=True)
cbar.formatter.set_powerlimits((0, 0))
cbar.update_ticks()

# Get automatically generated ticks
ticks = cbar.get_ticks()
new_ticks = (ticks + 0.00005)
cbar.set_ticks(new_ticks[1:-1])

plt.tick_params(axis='both', which='major', labelsize=tick_fontsize)
plt.tight_layout()

plt.show()

In [ ]:
width = cm_to_in(12)  # image width
height = cm_to_in(8) # image height

plt.figure(figsize=(height, height))
plt.scatter(u_prediction.flatten(), u_validation.flatten(), label='Prediction', color='red', s=5, alpha=0.3)
plt.scatter(u_validation.flatten(), u_validation.flatten(), label='Validation', color='black', s=5, alpha=0.3)

plt.xlabel('$u(x, y)$', fontsize=label_fontsize)
plt.ylabel('$u(x, y)$', fontsize=label_fontsize)

plt.grid(True)
# Get the current legend and change the order
handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 0]  # Change the order of the labels here (Validation first, Prediction second)
plt.legend(loc='upper left', fontsize=legend_fontsize)
plt.tick_params(axis='both', which='major', labelsize=tick_fontsize)

min = torch.min(u_validation.flatten()) - 0.1*(torch.max(u_validation.flatten() - torch.min(u_validation.flatten())))
max = torch.max(u_validation.flatten()) + 0.1*(torch.max(u_validation.flatten() - torch.min(u_validation.flatten())))

plt.xlim(min, max)
plt.ylim(0, 1.8)

plt.show()